### Simulate images to match experimental data and calibrate PT area with diffusivity

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../step3_Deep-SnapTrack/')) # or the path to your source code
sys.path.insert(0, module_path)

from pathlib import Path
import tifffile as tiff
import csv
import numpy as np
import torch
from os import path
import pandas as pd
from tqdm import tqdm
from ptorch_CNN_model import Model as buildModel

class IndividualNormalize(object):
    def __init__(self,upsampling_factor):
        self.upsampling_factor = upsampling_factor

    def __call__(self, sample):
        img = np.array(sample)

        # Define a function that projects an image to the range [0,1]
        img = np.squeeze(img)
        min_val = img.min()
        max_val = img.max()
        img = (img - min_val) / (max_val - min_val)

        # Normalize image given mean and std
        mean_img = img.mean()
        std_img = img.std()
        img = (img - mean_img) / std_img

        # Upsampling using a simple nearest neighbor interp.
        img = np.kron(img, np.ones((1, self.upsampling_factor, self.upsampling_factor)))
        
        return img
    
def simple_mask_cutoff(pred_img, min_threshold=0.1):
    # rescale image so that in the range [0 1]
    rescale_pred_img = (pred_img-pred_img.min())/(pred_img.max()-pred_img.min())

    # Find the index where the cumulative distribution first exceeds 
    # use contrast (Max and Min)
    pixel_threshold_index = rescale_pred_img >= min_threshold

    mask = np.zeros(pred_img.shape)

    # Get the corresponding pixel value
    mask[pixel_threshold_index] = 1

    area = mask.sum()

    return mask, area

# Define CUDA device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
f'Using device {device}'

'Using device cuda'

### Estimate the diffusivity based on simulated images

In [ ]:
# Define the directory of model weight
weights_file = Path('/path/to/trained_models/your_trained_model_name.pth')
# Define the directory containing your images
parent_path = Path('/path/to/save/simulated_dataset_forCalibrate_DiffCoeff')
img_path = parent_path.joinpath('imgs')
info_path = parent_path.joinpath('annotations')
csv_savepath = parent_path.joinpath('your_trained_model_name_PT_area_results.csv')

# Define normalization transformation
transform_img_norm = IndividualNormalize(upsampling_factor=10)

# Build the model
model = buildModel().to(device)
model.eval()

# Load the trained weights
model.load_state_dict(torch.load(weights_file))
f'Model loaded from {weights_file}'

# Get a list of all files in the directory and sort them
img_files = list(img_path.glob('*.img.tif'))

csv_filename = path.join(csv_savepath)
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['ImageName', 'frame', 'loc_id', 'simSPT_D', 'D_value', 'D_fromMSD', 'logD_fromMSD', 'exposure_time', 'molecule_class', 'x_pix_pos', 'y_pix_pos', 'SNR','background','noise','PT_area'])    

    for img_file in tqdm(img_files, total=len(img_files)):
        img = tiff.imread(img_file)
        info_file = info_path.joinpath( img_file.stem[:-4]+'.info.txt')
        info = pd.read_csv(info_file,sep='\t')
        total_frame_num = img.shape[0]
        for i in range(total_frame_num):
            img_norm = transform_img_norm(img[i])            
            # Convert the numpy array to a PyTorch tensor and move it to the CUDA device
            img_norm_tensor = torch.as_tensor(img_norm).float().unsqueeze(0).to(device)
            outputs = model(img_norm_tensor)

            outputs_cpu = outputs.cpu().detach().numpy()
            outputs_cpu = outputs_cpu.squeeze()

            # Threshold negative values
            outputs_cpu[outputs_cpu < 0] = 0
            mask, PT_area = simple_mask_cutoff(pred_img=outputs_cpu, min_threshold=0.1) # current best solution to estimate mobility                        

            writer.writerow([
                img_file.name,
                info.frame[i],
                info.loc_id[i],
                info.simSPT_D[i],
                info.D_value[i],
                info.D_fromMSD[i],
                info.logD_fromMSD[i],
                info.exposure_time[i],
                info.molecule_class[i],
                info.x_pix_pos[i],
                info.y_pix_pos[i],
                info.SNR[i],
                info.background[i],
                info.noise[i],
                PT_area
            ])                
        
print(f"CSV file '{csv_filename}' has been created.")


### Fitting on matlab curvefit app to obtain the constant
After obtain your_trained_model_name_PT_area_results.csv, read the csv into matlab using readtable() and fit using the equation y = a*sqrt(x), where y is PT area, and x is D_value, and obtain the fitting constant.